Now, I want to solve rest problem in a file and with RMHC,RMHC with a more powerful tweak and Steepest Step ane Restart.

Intance 4


In [1]:
import numpy as np
from icecream import ic
from tqdm import tqdm
import plotly.graph_objects as go

In [2]:
UNIVERSE_SIZE = 100000
NUM_SETS = 10000
DENSITY = 0.1

In [3]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

In [4]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

def tweak(solution):
  new_solution = solution.copy()
  flip_set = np.random.randint(0,NUM_SETS)
  new_solution[flip_set] = not new_solution[flip_set]
  return new_solution

In [5]:
def RMHC(SETS,COSTS,RESTARTS,STEPS):
  best_solution = None
  best_cost = float('inf')# biggest(CAN NOT BE 0! DUMB PROBLEM I MADE!)
  history = []

  for _ in tqdm(range(RESTARTS)):
    #random initial solution
    current_solution = np.random.choice([True, False], size=NUM_SETS, p=[0.5,0.5])
    current_cost = cost(current_solution)

    for _ in tqdm(range(STEPS)):
      # new_solution form tweak
      new_solution = tweak(current_solution)
      new_cost = cost(new_solution)

      #choose new solution if its better
      if valid(new_solution) and new_cost < current_cost:
        current_solution = new_solution
        current_cost = new_cost
        
    if best_cost > current_cost:
      best_solution = current_solution
      best_cost = current_cost

    history.append(best_cost)

  return best_solution,best_cost,history

In [6]:
best_solution,best_cost,history = RMHC(SETS,COSTS,RESTARTS=10,STEPS=100)

100%|██████████| 10/10 [02:04<00:00, 12.45s/it]


In [7]:
print("Best solution found with cost:", best_cost)

Best solution found with cost: 120995728.88901341


In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history, mode='lines', name='Cost History'))
fig.update_layout(title='Cost History of RMHC', xaxis_title='Iterations', yaxis_title='Cost')
fig.show()

Instance 5

In [9]:
DENSITY = 0.2

In [10]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

In [11]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

def Ptweak(solution):
  new_solution = solution.copy()
  flip_set = np.random.choice(NUM_SETS, size=np.random.randint(1, NUM_SETS // 10), replace=False)
  for index in flip_set:
        new_solution[index] = not new_solution[index]
  return new_solution

In [12]:
def RMHC(SETS,COSTS,RESTARTS,STEPS):
  best_solution = None
  best_cost = float('inf')# biggest(CAN NOT BE 0! DUMB PROBLEM I MADE!)
  history = []

  for _ in tqdm(range(RESTARTS)):
    #random initial solution
    current_solution = np.random.choice([True, False], size=NUM_SETS, p=[0.5,0.5])
    current_cost = cost(current_solution)

    for _ in tqdm(range(STEPS)):
      # new_solution form tweak
      new_solution = tweak(current_solution)
      new_cost = cost(new_solution)

      #choose new solution if its better
      if valid(new_solution) and new_cost < current_cost:
        current_solution = new_solution
        current_cost = new_cost
        
    if best_cost > current_cost:
      best_solution = current_solution
      best_cost = current_cost

    history.append(best_cost)

  return best_solution,best_cost,history

In [13]:
best_solution,best_cost,history = RMHC(SETS,COSTS,RESTARTS=10,STEPS=100)

100%|██████████| 10/10 [02:03<00:00, 12.34s/it]


In [14]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history, mode='lines', name='Cost History'))
fig.update_layout(title='Cost History of RMHC', xaxis_title='Iterations', yaxis_title='Cost')
fig.show()

Instacne 6

In [15]:
from itertools import accumulate

In [16]:
DENSITY = 0.3

In [17]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

In [18]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

def tweak(solution):
  new_solution = solution.copy()
  flip_set = np.random.randint(0,NUM_SETS)
  new_solution[flip_set] = not new_solution[flip_set]
  return new_solution

#I think its efficient and concise cost calculation,T = 1,F = 0
def evaluate(solution):
    return np.dot(solution, COSTS)

In [19]:
def SSR(SETS,COSTS,RESTART,STEP):
  best_solution = None
  best_cost = float('inf')
  history = []

  for _ in tqdm(range(RESTART),desc='Restarts'):
    current_solution = np.random.choice([True, False], size=NUM_SETS,p = [0.5,0.5])
    

    # To make sure the algorithm is able to do multiple restarts 
    # and every restart got enough step(? or move) to search solution
    for step in tqdm(range(STEP // RESTART),desc= 'Steps'):

      candidates =  [tweak(current_solution.copy())for _ in range(5)]
      candidates_cost = [evaluate(candidate) for candidate in candidates]

      #find the best solution(min cost)
      best_candidate_set = np.argmin(candidates_cost)
      best_candidate = candidates[best_candidate_set]
      best_candidate_cost = candidates_cost[best_candidate_set]

      history.append(best_candidate_cost)

      #choose best solution and update
      if best_candidate_cost < evaluate(current_solution):
        current_solution = best_candidate
        
      
      

    if evaluate(current_solution) < best_cost:
      best_solution = current_solution
      best_cost = evaluate(current_solution)

  return best_solution,best_cost,history


In [20]:
best_solution,best_cost,history = SSR(SETS,COSTS,100,3000)

Restarts: 100%|██████████| 100/100 [00:00<00:00, 225.76it/s]


In [21]:
print(f"Best solution cost : {evaluate(best_solution)},{best_solution}")

Best solution cost : 408732944.37617624


In [22]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(history))), y=list(accumulate(history, min)), mode='lines', name='Best Cost'))
fig.add_trace(go.Scatter(x=list(range(len(history))), y=history, mode='markers', name='Cost per Step'))
fig.update_layout(title="Cost Progression over Time (Steepest Step and Restart)", xaxis_title="Steps", yaxis_title="Cost")
fig.show()